In [15]:
ROOT_DIR = r"C:\Users\wyattluke.lowery\Documents\GitHub\laplib"

BUNNY_PATH = ROOT_DIR+ r"\StanfordBunny\lap1.mat"
BUNNY_PLY = ROOT_DIR + r"\StanfardBunny\src\Stanford Bunny\reconstruction\bun_zipper.ply"


from gridwb import *

import numpy as np
import scipy.sparse as sp
import os
import scipy.io as sio

def inspect_ply(file_path):
    M = Mesh.from_ply(file_path )
    B = M.get_incidence_matrix()

    num_vert = len(M.vertices)
    num_edge = B.shape[1]

    print(f"Verticies: {num_vert}")
    print(f"Edges: {num_edge}")

def fomrat_edges(M):
    
    B: sp.csc_matrix = M.get_incidence_matrix()

    B[B<0] = 0

    B.eliminate_zeros()
    B = B.tocoo()

    edges = np.stack([B.row, B.col, B.data], axis=1)

    return edges

# This function generates the edges.txt file

def write_edges_txt(filepath, edge_matrix, feature_names=None):
    """
    Writes an edge list to a text file adhering to the laplib edges.txt format.

    Format Specification:
    - Space-separated text file with a header.
    - Columns: source, target, followed by K feature columns.
    - Indices: 0-based integers.

    Args:
        filepath (str): Destination file path (e.g., 'Dataset/edges.txt').
        edge_matrix (numpy.ndarray or list): Matrix of shape (M, 2 + K).
                                             Col 0: Source index (0-based int)
                                             Col 1: Target index (0-based int)
                                             Col 2+: Edge features
        feature_names (list[str], optional): Names for the K feature columns.
                                             If None, defaults to ['feature_1', ...].
    """
    data = np.array(edge_matrix)
    
    # Validate dimensions
    if data.ndim != 2 or data.shape[1] < 2:
        raise ValueError("Input matrix must be M x (2+K) with at least source and target columns.")

    num_features = data.shape[1] - 2

    # Handle feature names
    if feature_names is None:
        feature_names = [f"feature_{i+1}" for i in range(num_features)]
        
    if len(feature_names) != num_features:
        raise ValueError(f"Provided {len(feature_names)} names for {num_features} feature columns.")

    # Construct header
    header = ["source", "target"] + feature_names

    with open(filepath, 'w') as f:
        # Write Header
        f.write(" ".join(header) + "\n")
        
        # Write Rows
        for row in data:
            # Enforce integer formatting for source and target indices
            src, tgt = int(row[0]), int(row[1])
            
            # Format features (preserves float formatting from numpy)
            features = row[2:]
            
            if len(features) > 0:
                features_str = " ".join(map(str, features))
                f.write(f"{src} {tgt} {features_str}\n")
            else:
                f.write(f"{src} {tgt}\n")

# This generates the signals.txt file


def write_signals_txt(filepath, signal_matrix, signal_names=None):
    """
    Writes a vertex signal matrix to a text file adhering to the laplib signals.txt format.

    Format Specification:
    - Space-separated text file.
    - Represents an N x D matrix where N is the number of vertices and D is the number of features.
    - Rows correspond to vertices sorted by index (0 to N-1).

    Args:
        filepath (str): Destination file path (e.g., 'Dataset/signals.txt').
        signal_matrix (numpy.ndarray or list): Matrix of shape (N, D).
        signal_names (list[str], optional): Names for the D signal columns.
                                            If provided, a header line is written.
    """
    data = np.array(signal_matrix)
    
    # Ensure 2D array (N x D). If 1D input, treat as N x 1.
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    
    if data.ndim != 2:
        raise ValueError("Input data must be a 2D matrix (N x D).")

    num_cols = data.shape[1]

    # Validate signal names if provided
    if signal_names is not None:
        if len(signal_names) != num_cols:
            raise ValueError(f"Provided {len(signal_names)} names for {num_cols} columns.")

    with open(filepath, 'w') as f:
        # Write Header if names are provided
        if signal_names:
            f.write(" ".join(signal_names) + "\n")
        
        # Write Rows
        for row in data:
            # Join row elements with spaces
            # map(str, row) handles both float and int types correctly
            f.write(" ".join(map(str, row)) + "\n")

Generate the `edges.txt` and `signals.txt` file

In [18]:
M = Mesh.from_ply(BUNNY_PLY)

write_edges_txt(
    "edges.txt",
    fomrat_edges(M),
    ['weight']
)


write_signals_txt(
    "signals.txt",
    M.get_xyz(),
    ['x', 'y', 'z']
)